In [1]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("data_utf8.csv")

# Convert 'date' column to datetime and extract month, day, and year
df["month"] = pd.to_datetime(df["date"]).dt.strftime('%b')  # E.g., Jan, Feb
df["day"] = pd.to_datetime(df["date"]).dt.day               # Day as a number (1-31)
df["year"] = pd.to_datetime(df["date"]).dt.year             # Year as a four-digit number


# Save the updated DataFrame back to the same file
df.to_csv("new_data.csv", index=False)

C:\Users\hp\AppData\Local\Temp\ipykernel_24516\1132825955.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_utf8.csv")


In [2]:
# imputation

#first remove unnecessary columns
df = df.drop(['stn_code', 'sampling_date', 'date', 'location_monitoring_station', 'pm2_5', 'agency'], axis=1)
# axis=1: Indicates that you're dropping columns (not rows)
print(df.head())
print(df.isnull().sum())

# Remove rows where all three columns 'month', 'day', and 'year' are missing
df = df.dropna(subset=['month', 'day', 'year'], how='all')
print(df.isnull().sum())

#  Impute 'rspm' based on the mean of 'rspm' for each month
month_means = df.groupby('month')['rspm'].transform('mean')
df['rspm'] = df['rspm'].fillna(month_means)

month_means1 = df.groupby('month')['spm'].transform('mean')
df['spm'] = df['spm'].fillna(month_means1)

month_means2 = df.groupby('month')['so2'].transform('mean')
df['so2'] = df['so2'].fillna(month_means2)

month_means3 = df.groupby('month')['no2'].transform('mean')
df['no2'] = df['no2'].fillna(month_means3)

# for locations missing:
#  Calculate the mode of 'location' for each 'state'
state_mode = df.groupby('state')['location'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

#  Impute missing 'location' values using the mode for the corresponding 'state'
df['location'] = df.apply(
    lambda row: state_mode[row['state']] if pd.isnull(row['location']) else row['location'], axis=1
)
# for type missing:
#  Calculate the mode of 'type' for each 'state'
location_mode = df.groupby('state')['type'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

#  Impute missing 'type' values using the mode for the corresponding 'state'
df['type'] = df.apply(
    lambda row: location_mode[row['state']] if pd.isnull(row['type']) else row['type'], axis=1   
)

"""
groupby('state')['location'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan):
This groups the data by state and calculates the mode of location for each state. If there are no valid modes (e.g., all NaN values), it returns NaN.
apply(lambda row: state_mode[row['state']] if pd.isnull(row['location']) else row['location'], axis=1):
This applies a function across each row.
If location is missing (NaN), it uses the mode of location for the corresponding state.
If location is not missing, it keeps the original value.
"""

print(df.isnull().sum())



            state   location                                type  so2   no2  \
0  Andhra Pradesh  Hyderabad  Residential, Rural and other Areas  4.8  17.4   
1  Andhra Pradesh  Hyderabad                     Industrial Area  3.1   7.0   
2  Andhra Pradesh  Hyderabad  Residential, Rural and other Areas  6.2  28.5   
3  Andhra Pradesh  Hyderabad  Residential, Rural and other Areas  6.3  14.7   
4  Andhra Pradesh  Hyderabad                     Industrial Area  4.7   7.5   

   rspm  spm month  day    year  
0   NaN  NaN   Feb  1.0  1990.0  
1   NaN  NaN   Feb  1.0  1990.0  
2   NaN  NaN   Feb  1.0  1990.0  
3   NaN  NaN   Mar  1.0  1990.0  
4   NaN  NaN   Mar  1.0  1990.0  
state            0
location         3
type          5393
so2          34646
no2          16233
rspm         40222
spm         237387
month            7
day              7
year             7
dtype: int64
state            0
location         0
type          5390
so2          34643
no2          16230
rspm         40219
spm 

In [3]:
from sklearn.preprocessing import LabelEncoder



categorical_columns = ['state', 'location', 'type', 'month', 'day', 'year']
label_encoders = {col: LabelEncoder() for col in categorical_columns}

for col, encoder in label_encoders.items():
    df[col] = encoder.fit_transform(df[col])
print(df.head())
df.shape


   state  location  type  so2   no2        rspm         spm  month  day  year
0      0       114     6  4.8  17.4  117.076940  239.597104      3    0     3
1      0       114     1  3.1   7.0  117.076940  239.597104      3    0     3
2      0       114     6  6.2  28.5  117.076940  239.597104      3    0     3
3      0       114     6  6.3  14.7  115.227345  237.053027      7    0     3
4      0       114     1  4.7   7.5  115.227345  237.053027      7    0     3


(435735, 10)

In [4]:
from sklearn.model_selection import train_test_split

# Features and target
X = df[['so2', 'no2', 'spm', 'state', 'location', 'type', 'month', 'day', 'year']]  #dropping some columns above has made its R -squared to 0.30 (was lesser before)
y = df['rspm']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)





In [5]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)



#now 
MAE = mean_absolute_error(y_test, y_pred)
print("MAE : ", MAE)

MSE = mean_squared_error(y_test, y_pred)
print("MSE : ", MSE)

#calculate R-squared of regression model
R_squared = model.score(X_train, y_train)
print("R_Squared : ", R_squared)


"""
Steps to increase R-squared
Remove Outliers: Identify and handle outliers using techniques like the IQR method or z-scores, as they can distort the relationship between variables. (Removing outliers will depreciate teh purpose of project. which is to be able to study the daat_monitoring faults/ anomalies study, etc.)
Impute Missing Values: Address missing data through mean, median, mode, or advanced imputation methods like KNN or regression-based imputation. (have done that)
Increase Dataset Size: A larger dataset often provides more robust estimates of relationships between variables. (already taking complete data)
"""

MAE :  41.454033732092405
MSE :  3498.536016625363
R_Squared :  0.30183803475590665


'\nSteps to increase R-squared\nRemove Outliers: Identify and handle outliers using techniques like the IQR method or z-scores, as they can distort the relationship between variables. (Removing outliers will depreciate teh purpose of project. which is to be able to study the daat_monitoring faults/ anomalies study, etc.)\nImpute Missing Values: Address missing data through mean, median, mode, or advanced imputation methods like KNN or regression-based imputation. (have done that)\nIncrease Dataset Size: A larger dataset often provides more robust estimates of relationships between variables. (already taking complete data)\n'